In [6]:
# import packages
import pandas as pd
import numpy as np
import re

In [5]:
#create dataframes

course_path = 'C:/Users/ywa/Documents/Data/Moodle Query/May2025/Course - Category - Enrollments.csv' 
cert_path = 'C:/Users/ywa/Documents/Data/Moodle Query/May2025/Course - Certificates.csv'

course_df = pd.read_csv(course_path)
cert_info = pd.read_csv(cert_path)
#course_df.head()
cert_info.head()


,certificateName,courseID,amount
0,Leistungsnachweis mit Vorlage,2,1
1,Leistungsnachweis,3,1
2,Record of Achievement,6,64
3,Confirmation of Participation,6,77
4,Teilnahmebescheinigung,7,2


In [7]:
# groupby cert_info by courseID
participation_key = r'teilnahmebestätigung|participation'
achievement_key = 'leistungsnachweis|achivement|record of achievement|zertifikat'

cert_info['name_lower'] = cert_info['certificateName'].str.lower()

def classify_cert(name):
    if pd.isna(name):
        return None
    name = name.lower()
    if re.search(participation_key, name):
        return 'CoP'
    if re.search(achievement_key, name):
        return 'RoA'
    else:
        return None

cert_info['cert_type'] = cert_info['name_lower'].apply(classify_cert)

cert_info_filtered = cert_info[cert_info['cert_type'].notna()]

cert_df = (cert_info_filtered.groupby(['courseID', 'cert_type'])['amount'].sum().unstack('cert_type').reset_index())

cert_df.fillna(0, inplace = True)

cert_df[['CoP', 'RoA']] = cert_df[['CoP', 'RoA']].astype(int)

cert_df.columns = ['courseID', 'comfirmation_of_participation', 'records_of_achievement']

print(cert_df.shape[0])

cert_df.head()


108


,courseID,comfirmation_of_participation,records_of_achievement
0,2,0,1
1,3,0,1
2,6,77,64
3,7,0,1
4,10,123,109


In [11]:
# merge two dataframes
course_df = course_df[['courseID', 'courseName', 'categoryName', 'enrollments']]

print(len(course_df))
print(len(cert_df))

merged_df = course_df.merge(cert_df, on = 'courseID', how = 'left')

print(len(merged_df))

merged_df.head()

166
108
166


,courseID,courseName,categoryName,enrollments,comfirmation_of_participation,records_of_achievement
0,106,Einführung in die KI,Über KI,7738,1086.0,823.0
1,19,Sprachassistenzen als Chance für die Hochschul...,Erwachsenenbildung,3652,514.0,460.0
2,58,AICE your exams – Generative KI als Copilot im...,Studieren,2767,573.0,105.0
3,99,KI für Alle 1: Einführung in die Künstliche In...,Über KI,2761,202.0,119.0
4,27,Identifikation geeigneter KI-Anwendungsfälle,Industrie 4.0,2691,183.0,185.0


In [12]:
# export data for spreadsheet check
#merged_df.to_excel('C:/Users/ywa/Documents/Data/Moodle Query/May2025/MoodleEnrole.xlsx', index = False)